# Save and Load Models

In this notebook we give examples regarding the saving and loading of `NeuralForecast` models.

The two methods to consider are:<br>
1. `NeuralForecast.save`: Saves models into disk, allows save dataset and config.<br>
2. `NeuralForecast.load`: Loads models from a given path.<br>

You can run these experiments using GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/neuralforecast/blob/main/nbs/examples/Save_Load_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installing NeuralForecast

In [ ]:
%%capture
!pip install git+https://github.com/nixtla/neuralforecast.git

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from ray import tune

from neuralforecast.core import NeuralForecast
from neuralforecast.auto import AutoMLP
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.utils import AirPassengersDF

## 2. Loading AirPassengers Data

In [ ]:
Y_df = AirPassengersDF
Y_df = Y_df.reset_index(drop=True)
Y_df.head()

,unique_id,ds,y
0,1.0,1949-01-31,112.0
1,1.0,1949-02-28,118.0
2,1.0,1949-03-31,132.0
3,1.0,1949-04-30,129.0
4,1.0,1949-05-31,121.0


## 3. Model Training

In [ ]:
# Split train/test sets
Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31']   # 12 test

In [ ]:
%%capture
horizon = len(Y_test_df)
models = [NBEATS(input_size=2 * horizon, h=horizon, max_epochs=50),
          NBEATS(input_size=3 * horizon, h=horizon, max_epochs=50),
          NHITS(input_size=2 * horizon, h=horizon, max_epochs=50),
          AutoMLP(# Ray tune explore config
                  config=dict(max_steps=100, # Operates with steps not epochs
                              input_size=tune.choice([3*horizon]),
                              learning_rate=tune.choice([1e-3])),
                  h=12,
                  num_samples=1, cpus=1)]
fcst = NeuralForecast(models=models, freq='M')

In [ ]:
%%capture
fcst.fit(df=Y_train_df)

In [ ]:
Y_hat_df = fcst.predict().reset_index()
Y_hat_df = Y_test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])
plot_df = pd.concat([Y_train_df, Y_hat_df]).set_index('ds')

fig, ax = plt.subplots(1, 1, figsize = (20, 7))
plot_df[['y', 'NBEATS', 'NBEATS1', 'NHITS', 'AutoMLP']].plot(ax=ax, linewidth=2)

ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()

## 4. Save and Load Models

In [ ]:
fcst.save(path='./checkpoints/test_run/', model_index=None, 
          overwrite=True, save_dataset=True)

In [ ]:
fcst2 = NeuralForecast.load(path='./checkpoints/test_run/')

In [ ]:
Y_hat_df = fcst2.predict().reset_index()
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
Y_hat_df = Y_test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])
plot_df = pd.concat([Y_train_df, Y_hat_df]).set_index('ds')

plot_df[['y', 'NBEATS', 'NBEATS1', 'NHITS', 'MLP']].plot(ax=ax, linewidth=2)

ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()